In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-12-13"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
23762,2024-12-13,Geórgia Superliga,12:00,Gurjaani Delta,Iverioni,1.44,2.52,164.5,1.88,1.79,-6.5,1.95,1.72,h06ECE55,0.694444,0.396825,0.531915,0.558659,0.091270,131.236,21.069937,0.160550,1.2,1.643168,1.369306,120.40,1.666,0.280054,0.168099,-7.0,169.896,48.605068,0.286087,1.2,1.643168,1.369306,173.74,1.992,0.527987,0.265054,-35.0,86,73,1.40,2.38,118.128,183.294,0.385695,0.034681,0.088629,-3.62,-0.724,-0.607735,0.704723,0.4,-0.304723,0.25,0.050,30.400000,0.000000,0.0,0.000000
23763,2024-12-13,Islândia Premier League,16:00,Thorl,Alftanes,1.72,1.95,182.5,1.83,1.83,-2.5,1.99,1.70,44ZhmZqC,0.581395,0.512821,0.546448,0.546448,0.094216,173.020,101.455002,0.586377,1.8,1.643168,0.912871,149.46,2.104,1.321923,0.628290,-37.0,194.480,100.759388,0.518096,1.8,1.643168,0.912871,190.96,2.186,1.025978,0.469340,-7.0,106,77,1.41,2.48,138.942,210.384,0.088629,0.000000,0.111144,0.83,0.166,4.337349,0.693951,0.6,-0.093951,-1.14,-0.228,-4.166667,0.517605,0.4,-0.117605
23764,2024-12-13,Islândia Premier League,16:30,Grindavik,Valur,1.29,3.15,179.5,1.86,1.80,-8.5,1.96,1.72,vJP35Gat,0.775194,0.317460,0.537634,0.555556,0.092654,168.030,38.852771,0.231225,1.8,1.643168,0.912871,202.80,1.670,0.382557,0.229076,57.0,124.290,31.429970,0.252876,0.6,1.341641,2.236068,106.70,1.396,0.276279,0.197907,-21.0,104,97,1.95,1.10,134.814,154.494,0.592441,0.023184,0.092231,0.65,0.130,2.230769,0.695589,0.9,0.204411,-3.09,-0.618,-3.478964,0.527693,0.3,-0.227693
23765,2024-12-13,Israel Liga Leumit,07:15,Elitzur Shomron,Elitzur Ashkelon,2.62,1.41,175.5,1.83,1.83,4.5,1.97,1.71,GWCwlJxI,0.381679,0.709220,0.546448,0.546448,0.090899,237.038,95.819629,0.404237,0.0,0.000000,NaN,299.52,2.806,1.169991,0.416961,-46.0,236.106,93.845568,0.397472,3.0,0.000000,0.000000,109.62,2.462,0.809086,0.328630,29.0,78,87,3.84,1.26,149.616,261.426,0.424615,0.000000,0.099917,-3.05,-0.610,-2.655738,0.604313,0.4,-0.204313,3.49,0.698,0.587393,0.459203,0.3,-0.159203
23766,2024-12-13,Portugal Proliga,18:30,Braga,Illiabum,1.27,3.26,149.5,1.88,1.78,-8.5,1.88,1.78,r15fqXUG,0.787402,0.306748,0.531915,0.561798,0.094150,110.574,11.385020,0.102963,3.0,0.000000,0.000000,112.10,1.282,0.223428,0.174281,92.0,94.534,16.194315,0.171307,1.8,1.643168,0.912871,85.68,1.168,0.074967,0.064184,80.0,95,72,1.18,1.19,139.368,133.574,0.621255,0.038640,0.038640,-1.16,-0.232,-1.163793,0.698447,0.5,-0.198447,-2.65,-0.530,-4.264151,0.619881,0.7,0.080119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23869,2024-12-13,China Cba,08:35,Zhejiang Guangsha,Fujian,1.05,7.16,213.5,1.88,1.78,-16.5,1.96,1.72,SdoyH9qC,0.952381,0.139665,0.531915,0.561798,0.092046,127.880,25.602225,0.200205,3.0,0.000000,0.000000,98.58,1.256,0.266889,0.212492,57.0,647.972,438.010470,0.675971,0.0,0.000000,NaN,348.60,6.288,4.258987,0.677320,-84.0,93,105,1.06,3.32,123.074,697.488,1.052478,0.038640,0.092231,1.29,0.258,0.193798,0.768986,0.9,0.131014,-5.00,-1.000,-6.160000,0.193368,0.0,-0.193368
23870,2024-12-13,China Cba,08:35,Tianjin,Zhejiang Chouzhou,5.67,1.09,224.5,1.80,1.86,11.5,2.03,1.67,pYH1XUTh,0.176367,0.917431,0.555556,0.537634,0.093798,332.590,151.374712,

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
23763,16:00,Islândia Premier League,Thorl,Alftanes,1.72,Back Home
23772,16:30,França Pro B,St. Chamond,Rouen,1.66,Back Home
23779,13:30,Finlândia I Divisão A,Lahti Basketball,Helsingin NMKY,1.83,Back Home
23816,14:00,Europa Liga Letônia,Ventspils,BK Ogre,1.56,Back Home
23820,05:30,Austrália Nbl,Melbourne United,Illawarra Hawks,1.54,Back Home
23829,17:00,Europa Euroliga,Lyon-Villeurbanne,Paris,2.28,Back Home
23860,07:00,Coreia Do Sul Kbl,Anyang JungKwanJang,LG Sakers,1.98,Back Home
